<h1>Machine learning using Neural networks to predict eye state data</h1>
<p>
The purpose of this project is to get more familiar with an eeg dataset and learn more about data manipulation. The dataset is slipt into 15 differents columns with the last being the target values (0,1). The target values represent the state of the eye, being open or closed. The other data represents the value of an eeg node which gets the value of the electrical activity in that sector surrounding the node.<br>
This dataset is located in the UCI database found <a href='https://archive.ics.uci.edu/ml/datasets/EEG+Eye+State'>here</a>
</p>

<h2>The purpose of the nodes and locations</h2>
<p>The nodes used in this dataset are located in the red filled nodes shown below in Figure 1.</p>
<img src='data/eeg_eye_state_map.png' alt='eeg_eye_state_map'>Figure 1 <br>Source:
<a href='https://www.researchgate.net/figure/Surface-map-of-EEG-electrode-locations_fig34_230864997'>https://www.researchgate.net/figure/Surface-map-of-EEG-electrode-locations_fig34_230864997</a>

<h2>The purpose of the node locations</h2>
<p>The node locations are placed uniformly on the scalp in order to get the maximum coverage of the brain with the most uniform data. The barin is composed of different reigons, namely the:
    <ul>
        <li>Occipital lobe: Visual area, sight, image recoginition</li>
        <li>Temporal lobe: Association area, short term memory, emotion</li>
        <li>Parential lobe: Motor function, voluntary muscles</li>
        <li>Frontal lobe: Brocas area, muscles of speech</li>
        <li>Temporal lobe: Auditory area, hearing</li>
        <li>[name]: Emotional area, pain, hunger, 'flight or fight'</li>
        <li>Pariental lobe: Sensory association</li>
        <li>Frontal lobe: Oilfactory, smelling</li>
        <li>Pariental lobe: Sensory area, sensation grom skin and muscles</li>
        <li>Pariental lobe: Somatosensory association, evaluation of weight, temperature, texture (object recognition)</li>
        <li>Temporal lobe: Wernicke's area, Written and spoken language comprehension</li>
        <li>Cerebral Cortex: Motor function, eye movement and orientation</li>
        <li>Frontal lobe: Higher mental functions, concentration, planning, judgement, emotional expression, creativity, inhibition</li>
        <li>Cerebellum: Motor functions, balance, equilibrium, posture, coordination of movement</li>
    </ul>
    The data above can be visualized in Figure 2.
</p>
<img src='data/anatomy-function-brain-areas-basics-large.jpg' alt='Brain location map'>
Figure 2<br>Source:
<a href='https://www.dana.org/wp-content/uploads/2019/08/anatomy-function-brain-areas-basics-large.jpg'>https://www.dana.org/wp-content/uploads/2019/08/anatomy-function-brain-areas-basics-large.jpg</a></img>
<p>By looking at the node locations in this experiment and comparing with the lobe location and function we can see why these certain locations were picked.
    <ul>
        <li>AF3/AF4: Frontal lobe- Higher mental functions</li>
        <li>F3/F4: Frontal lobe- Higher mental functions</li>
        <li>F7/F8: Inferior Frontal/Brocas- Functions of Speech</li>
        <li>FC5/FC8: Cerebral Cortex-Motor functions <strong>eye movement/ orientation</strong></li>
        <li>T7/T8: Temporal lobe- Short term memory/emotion</li>
        <li>P7/P8: Posterior Temporal- Hearing, speech <strong>visual</strong> recoginition</li>
        <li>O1/O2: Occipital lobe- <strong>Visual, sight, image recognition</strong></li>
    </ul>
    Looking at the locations of the nodes placement can help us gain an understanding of the <i>why?</i> The nodes seemed to be placed uniformly. (Which is debateably important. See: <a href='https://www.health.harvard.edu/blog/right-brainleft-brain-right-2017082512222'>https://www.health.harvard.edu/blog/right-brainleft-brain-right-2017082512222</a>) Looking further into the placement the nodes seem to cover the lobes of the brain that have to do with vision, along with some other lobes pertaining to thought/ memory and speech.
</p>

<h2>Code: EEG eye-state prediction</h2>

In [1]:
# import libraries
import os
import time
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff
from scipy import stats

from keras.utils import normalize
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import tensorflow.keras

Using TensorFlow backend.


<p>Prepare tensorboard</p>

In [2]:
# home=os.path()
home='C:/Users/einel/Documents/Python_projects/EEG Eye State data set'
root_logdir = os.path.join(os.curdir, "my_logs")
print(root_logdir) # The "." represents your current directory

.\my_logs


In [3]:
def get_run_logdir(root_logdir):
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id), run_id

In [4]:
# Load in the data frame
data_frame=home+'/data/EEG Eye State.arff'
raw_data = loadarff(data_frame)
df_data = pd.DataFrame(raw_data[0])
# df_data.head()

<p>In order to pre-process the data I split the data into two sets, The data and the targets. Next I normalized by z-score to make the data more manageable and easier to learn.<br>The targets here are 1 or 0, meaning is the eye open or closed?</p>

In [5]:
# Preprocess Data
# split into data and targets
data_targets=df_data.iloc[:,-1]
data=df_data.iloc[:,:-1]

# data normalization with sklearn z-score
scaler = StandardScaler()
scaler.fit(data)
data = scaler.transform(data)

# convert targets into 1 or 0
data_targets=np_utils.to_categorical(data_targets, num_classes=2)

<p>In order to split into training and test data I use the method train_test_split() which randomizes the data while splitting it into a training and testing data set. The data is split into 80% training and 20% testing and validation.</p>

In [6]:
# split the data
xTrain, xTest, yTrain, yTest = train_test_split(data, data_targets, test_size = 0.2)
# print(xTrain, yTrain)

<h2>Basic classification</h2>
<p>This is the creation of the model.<br>
The model is split into 4 layers with two being the hidden layers. The goal here is to make a network that can accurately predict the state of the eye, being 0 or 1.<br>
</p>

In [7]:
# prepare save files
run_logdir, run_id = get_run_logdir(root_logdir)
print(run_logdir)
tensorboard_cb = tensorflow.keras.callbacks.TensorBoard(run_logdir)
# create model
model = Sequential()
model.add(Dense(500, input_dim=14))
model.add(Dense(200, activation='relu'))
model.add(Dense(200, activation='tanh'))
model.add(Dense(2, activation='sigmoid'))
print(model.summary())

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x=xTrain, y=yTrain,epochs=200,batch_size=32, validation_data=(xTest,yTest), callbacks=[tensorboard_cb])

score, acc = model.evaluate(x=xTest, y=yTest)
# print(model.predict(xTest))

print('Accuracy: ',100*(acc))

.\my_logs\run_2020_05_07-12_56_03
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               7500      
_________________________________________________________________
dense_2 (Dense)              (None, 200)               100200    
_________________________________________________________________
dense_3 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 402       
Total params: 148,302
Trainable params: 148,302
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 11984 samples, validat

Epoch 98/200
11984/11984 [==============================] - 1s 110us/step - loss: 0.0450 - accuracy: 0.9830 - val_loss: 0.1238 - val_accuracy: 0.9589
Epoch 99/200
11984/11984 [==============================] - 1s 116us/step - loss: 0.0382 - accuracy: 0.9864 - val_loss: 0.1435 - val_accuracy: 0.9496
Epoch 100/200
11984/11984 [==============================] - 2s 132us/step - loss: 0.0478 - accuracy: 0.9825 - val_loss: 0.1403 - val_accuracy: 0.9563
Epoch 101/200
11984/11984 [==============================] - 1s 110us/step - loss: 0.0426 - accuracy: 0.9855 - val_loss: 0.1508 - val_accuracy: 0.9484
Epoch 102/200
11984/11984 [==============================] - 1s 120us/step - loss: 0.0387 - accuracy: 0.9860 - val_loss: 0.1344 - val_accuracy: 0.9539
Epoch 103/200
11984/11984 [==============================] - 1s 116us/step - loss: 0.0408 - accuracy: 0.9842 - val_loss: 0.1669 - val_accuracy: 0.9448
Epoch 104/200
11984/11984 [==============================] - 5s 455us/step - loss: 0.0455 - accu

In [15]:
# back to home directory
print(home)
# save the model
new_dir=home+'/data/'+run_id+'.h5'
model.save(new_dir)

C:/Users/einel/Documents/Python_projects/EEG Eye State data set


<p>This is to run the Tensorboard:</p> <a href="http://localhost:6006">http://localhost:6006</a>

In [8]:
tensorboard --logdir logs/fit

SyntaxError: invalid syntax (<ipython-input-8-46a9c1cd2d11>, line 1)

<h2>Conclusion</h2>
<p>
    In all it is safe to say that EEG data is very complex, but it is trainable. There is a lot that goes into gathering EEG data, namely the position of the nodes. In this conclusion I was able to reach a 93.858% accuracy on the given data set.
</p>

<h2>Sources</h2>
<ul>
    <li>https://www.dana.org/article/neuroanatomy-the-basics/</li>
</ul>